In [1]:
import pandas as pd
import os

In [2]:
filel_path = "../data/raw/Fraudulent.csv"
df = pd.read_csv(filel_path)
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [4]:
df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [5]:
df.dropna(inplace=True)

In [6]:
df = df.drop_duplicates()

In [9]:
df = df.rename(columns={
    "step": "time_step",
    "type": "transaction_type",
    "amount": "amount",
    "nameOrig": "sender_id",
    "oldbalanceOrg": "sender_old_balance",
    "newbalanceOrig": "sender_new_balance",
    "nameDest": "receiver_id",
    "oldbalanceDest": "receiver_old_balance",
    "newbalanceDest": "receiver_new_balance",
    "isFraud": "is_fraud",
    "isFlaggedFraud": "is_flagged_fraud"
})


In [10]:
df.head()

,time_step,transaction_type,amount,sender_id,sender_old_balance,sender_new_balance,receiver_id,receiver_old_balance,receiver_new_balance,is_fraud,is_flagged_fraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [12]:
numeric_cols = [
    "time_step", "amount", 
    "sender_old_balance", "sender_new_balance",
    "receiver_old_balance", "receiver_new_balance",
    "is_fraud", "is_flagged_fraud"
]

df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column                Dtype  
---  ------                -----  
 0   time_step             int64  
 1   transaction_type      object 
 2   amount                float64
 3   sender_id             object 
 4   sender_old_balance    float64
 5   sender_new_balance    float64
 6   receiver_id           object 
 7   receiver_old_balance  float64
 8   receiver_new_balance  float64
 9   is_fraud              int64  
 10  is_flagged_fraud      int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [14]:
df["transaction_type"] = df["transaction_type"].astype("category")
df["sender_id"] = df["sender_id"].astype("string")
df["receiver_id"] = df["receiver_id"].astype("string")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column                Dtype   
---  ------                -----   
 0   time_step             int64   
 1   transaction_type      category
 2   amount                float64 
 3   sender_id             string  
 4   sender_old_balance    float64 
 5   sender_new_balance    float64 
 6   receiver_id           string  
 7   receiver_old_balance  float64 
 8   receiver_new_balance  float64 
 9   is_fraud              int64   
 10  is_flagged_fraud      int64   
dtypes: category(1), float64(5), int64(3), string(2)
memory usage: 491.5 MB


In [15]:
balance_cols = [
    "sender_old_balance", "sender_new_balance",
    "receiver_old_balance", "receiver_new_balance"
]

for col in balance_cols:
    df[col] = df[col].apply(lambda x: max(x, 0))
df.head()

,time_step,transaction_type,amount,sender_id,sender_old_balance,sender_new_balance,receiver_id,receiver_old_balance,receiver_new_balance,is_fraud,is_flagged_fraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [16]:
df["is_high_value"] = (df["amount"] > 10000).astype(int)
df.head()

,time_step,transaction_type,amount,sender_id,sender_old_balance,sender_new_balance,receiver_id,receiver_old_balance,receiver_new_balance,is_fraud,is_flagged_fraud,is_high_value
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0,1


In [17]:
df["sender_balance_diff"] = df["sender_old_balance"] - df["sender_new_balance"]
df["receiver_balance_diff"] = df["receiver_new_balance"] - df["receiver_old_balance"]
df.head()

,time_step,transaction_type,amount,sender_id,sender_old_balance,sender_new_balance,receiver_id,receiver_old_balance,receiver_new_balance,is_fraud,is_flagged_fraud,is_high_value,sender_balance_diff,receiver_balance_diff
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0,0,9839.64,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0,0,1864.28,0.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0,0,181.00,0.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0,0,181.00,-21182.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0,1,11668.14,0.0


In [18]:
df["is_receiver_merchant"] = df["receiver_id"].str.startswith("M").astype(int)


In [19]:
df = df[~(
    (df["amount"] > 0) &
    (df["sender_old_balance"] == df["sender_new_balance"]) &
    (df["receiver_old_balance"] == df["receiver_new_balance"])
)]

In [20]:
df.to_csv('../data/processed/Fraudulent_processed.csv', index=False)